# Unit 2 Integrating Multiple MCP Servers to an OpenAI Agent

## Connecting Agents to Multiple MCP Servers

Welcome back\! In the previous lesson, you learned how to make your agent more efficient by enabling **tool caching** when connecting to a single **MCP server**. This allowed your agent to quickly access the list of available tools, reducing latency and improving the user experience, especially during multi-turn conversations.

-----

Now, let’s take the next step in building more capable and autonomous agents. In real-world scenarios, you often need your agent to handle tasks that go beyond the scope of a single MCP server. For example, imagine an agent that not only manages a shopping list but can also place orders for products from suppliers. To achieve this, you need to connect your agent to multiple MCP servers, each providing a different set of tools.

In this lesson, you will learn how to set up and coordinate multiple MCP servers, allowing your agent to solve more complex tasks by using tools from different services — all within a single conversation.

### Revisiting The Basics Of MCP Server Integration

Before we dive into multi-server integration, let’s quickly revisit how an agent connects to a single MCP server. In the previous lesson, you saw that an MCP server is a process that exposes a set of tools — actions the agent can use to help answer user queries. The agent connects to the MCP server, fetches the list of available tools (optionally caching them), and then uses these tools as needed to fulfill user requests.

This single-server setup works well for focused tasks, such as managing a shopping list. However, as your agent’s responsibilities grow, you may want it to interact with several different services. For example, one MCP server might handle shopping list management, while another could handle product ordering. By connecting to multiple MCP servers, you can give your agent access to a much broader set of capabilities, making it more useful and autonomous.

### Launching And Configuring Multiple MCP Servers

To connect your agent to more than one MCP server, you can use the OpenAI Agents SDK’s **MCPServerStdio** class for local subprocess servers, the **MCPServerSse** class for remote HTTP-based servers, or even combine both types in a single agent. Each server instance should be configured individually, and you can assign a unique **name** to each for easier identification during debugging or logging.

For example, suppose you have two local MCP servers: one for managing a shopping list (`list/mcp_server.py`) and another for ordering products (`products/mcp_server.py`). You can launch both servers like this:

```python
async with MCPServerStdio(
    params={
        "command": "python",
        "args": ["list/mcp_server.py"],
    },
    name="Shopping List",
    cache_tools_list=True,
) as list_server, MCPServerStdio(
    params={
        "command": "python",
        "args": ["products/mcp_server.py"],
    },
    name="Order Products",
    cache_tools_list=True,
) as order_server:
    # ... use the servers here ...
```

Here, each `MCPServerStdio` instance is configured with the `command` to run the server, a descriptive **name**, and the `cache_tools_list` parameter set to `True` for efficiency.

### Building An Agent To Use Multiple MCP Servers

Once you have your MCP servers running, you can create an agent that uses tools from both servers. The key is to pass a list of MCP server instances to the agent’s `mcp_servers` property. The agent will automatically aggregate the tools from all connected servers, making them available for use in any order or combination.

Here’s how you can set up the agent:

```python
agent = Agent(
    name="Shopping Assistant",
    instructions=(
        "You are an assistant that uses shopping list and order products tools to help manage a shopping list and order products.\n"
        "Your main responsibilities include:\n"
        "- Managing and updating the shopping list\n"
        "- Ordering products from appropriate suppliers\n"
        "- Handling all tasks without asking for user confirmation\n"
        "- Making decisions autonomously based on the available tools"
    ),
    mcp_servers=[list_server, order_server],
    model="gpt-4.1"
)
```

In this example, the agent receives both the `list_server` and `order_server` instances. The **instructions** are crafted to guide the agent in using both sets of tools to manage the shopping list and place orders, all without needing to ask the user for confirmation. This setup allows the agent to coordinate actions across different services, making it much more powerful and autonomous.

To run the agent, you can use the `Runner.run` method and provide an input prompt:

```python
result = await Runner.run(
    starting_agent=agent,
    input="Order all the unpurchased items in the shopping list."
)
print(result.final_output)
```

A typical output might look like this:

```
All unpurchased items from the shopping list have been ordered:
- Bread and Coffee were ordered from the general supplier.
- Apples were ordered from the fruit supplier.
```

With this setup, the agent can check the shopping list using tools from one MCP server, then use tools from another MCP server to order each item from the correct suppliers. This allows the agent to coordinate actions across both servers in a single workflow, completing the entire process autonomously.

-----

### Summary And Next Steps

In this lesson, you learned how to connect an agent to multiple MCP servers, giving it access to a wider range of tools and capabilities. By configuring and launching each server separately and passing them to the agent, you enable your agent to coordinate actions across different services. This approach allows your agent to solve more complex, real-world tasks — such as managing a shopping list and ordering products — without needing user confirmation for every step.

You are now ready to practice these skills in hands-on exercises. Great job mastering this advanced integration technique\!

## Switching Agent’s MCP Server

You just learned how connecting to different MCP servers gives your agent new abilities. Now, let’s practice switching your agent from one server to another.

Your current code connects to a shopping list MCP server with instructions for managing a shopping list. Update it to connect to the ordering products MCP server instead, and adjust the agent’s instructions and input prompt to focus on ordering products.

To complete this exercise:

Modify the server configuration to use "products/mcp_server.py" and set name="Order Products".
Change the agent’s instructions so that they describe ordering products, not managing a shopping list.
Update the input prompt so that it asks the agent to order products instead of adding to a list.
This will help you become comfortable with changing server connections and adapting your agent’s behavior for different tasks.

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # TODO: Replace the shopping list server configuration with the ordering products server configuration
    async with MCPServerStdio(
        params={"command": "python", "args": ["list/mcp_server.py"]},
        name="Shopping List",
        cache_tools_list=True,
    ) as mcp_server:
        
        # TODO: Update the agent's instructions to focus on ordering products
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # TODO: Change the input prompt to order the items instead of adding them
        result = await Runner.run(
            starting_agent=agent,
            input="Add 10 strawberries"
        )

        # Print the agent's response
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # Replace the shopping list server configuration with the ordering products server configuration
    async with MCPServerStdio(
        params={"command": "python", "args": ["products/mcp_server.py"]}, # Changed to products/mcp_server.py
        name="Order Products", # Changed name to Order Products
        cache_tools_list=True,
    ) as mcp_server:
        
        # Update the agent's instructions to focus on ordering products
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses product ordering tools to help order products from suppliers.", # Updated instructions
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Change the input prompt to order the items instead of adding them
        result = await Runner.run(
            starting_agent=agent,
            input="Order 10 strawberries from the fruit supplier." # Updated input prompt
        )

        # Print the agent's response
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

## Integrating Multiple MCP Servers

Now, it’s your turn to set up an agent that can use both a shopping list server and an ordering products server at the same time.

To complete this exercise:

Add and configure the ordering products MCP server with stdio connection alongside the existing shopping list server. Make sure each server has a unique name and uses tool caching.
Update the agent’s instructions so they clearly describe both managing a shopping list and ordering products from suppliers.
Change the input prompt so it asks the agent to order all the unpurchased items in the shopping list, not just add a single item.
This will help you practice combining multiple MCP servers and adjusting your agent’s behavior for more complex tasks.

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # TODO: Add and configure the ordering products MCP server alongside the shopping list server
    async with MCPServerStdio(
        params={"command": "python", "args": ["list/mcp_server.py"]},
        name="Shopping List",
        cache_tools_list=True,
    ) as shopping_server:
        
        # TODO: Update the agent's instructions to cover both shopping list and ordering products tasks
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            mcp_servers=[shopping_server],
            model="gpt-4.1"
        )
        
        # TODO: Change the input prompt to ask the agent to order all products
        result = await Runner.run(
            starting_agent=agent,
            input="Add 10 strawberries"
        )

        # Print the agent's response
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

-----

## Integrating Multiple MCP Servers

Here's the corrected code that integrates both the shopping list and ordering products MCP servers, updates the agent's instructions, and modifies the input prompt to order all unpurchased items.

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # Add and configure the ordering products MCP server alongside the shopping list server
    async with MCPServerStdio(
        params={"command": "python", "args": ["list/mcp_server.py"]},
        name="Shopping List",
        cache_tools_list=True,
    ) as shopping_server, MCPServerStdio( # Added the ordering products server
        params={"command": "python", "args": ["products/mcp_server.py"]},
        name="Order Products",
        cache_tools_list=True,
    ) as order_server:
        
        # Update the agent's instructions to cover both shopping list and ordering products tasks
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that helps manage a shopping list and order products from suppliers. Use the shopping list tools to manage items, and the ordering tools to procure unpurchased items.", # Updated instructions
            mcp_servers=[shopping_server, order_server], # Both servers are now connected
            model="gpt-4.1"
        )
        
        # Change the input prompt to ask the agent to order all products
        result = await Runner.run(
            starting_agent=agent,
            input="Order all unpurchased items in the shopping list." # Changed input prompt
        )

        # Print the agent's response
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

## Mixing MCP Server Connection Types

Now, let’s take it further by mixing connection types—using a local server with stdio and a remote server with SSE—so your agent can work seamlessly with both.

Specifically, you need to:

Change the ordering products server connection from MCPServerStdio to MCPServerSse.
Set the url parameter for MCPServerSse to "http://localhost:7000/sse" (the server is already running at this URL).
This exercise will help you practice mixing local and remote MCP server connections in a single agent, a common requirement in real-world agent integration scenarios.


```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio, MCPServerSse


async def main():
    # TODO: Connect to the ordering server via SSE at http://localhost:7000/sse
    async with MCPServerStdio(
        params={"command": "python", "args": ["list/mcp_server.py"]},
        name="Shopping List",
        cache_tools_list=True,
    ) as shopping_server, MCPServerStdio(
        params={"command": "python", "args": ["products/mcp_server.py"]},
        name="Order Products",
        cache_tools_list=True,
    ) as order_server:
        
        # Create an agent with instructions for both servers
        agent = Agent(
            name="Shopping Assistant",
            instructions=(
                "You are an assistant that uses shopping list and order products tools to help manage a shopping list and order products.\n"
                "Your main responsibilities include:\n"
                "- Managing and updating the shopping list\n"
                "- Ordering products from appropriate suppliers\n"
                "- Handling all tasks without asking for user confirmation\n"
                "- Making decisions autonomously based on the available tools"
            ),
            mcp_servers=[shopping_server, order_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Order all the unpurchased items in the shopping list"
        )

        # Print the agent's response
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

Alright, my apologies for the missteps\! It seems I was overthinking the previous errors due to the confusing nature of the tracebacks. Thank you for the clear feedback on how `MCPServerSse` and `MCPServerStdio` should be configured.

It's helpful to know:

  * `MCPServerSse` *does* expect a `params` dictionary with a `"url"` key.
  * `MCPServerStdio` *does NOT* need a `"url"` in its `params`. My attempt to add it was indeed incorrect and caused an error.

Let's apply these corrections and give it another shot.

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio, MCPServerSse


async def main():
    async with MCPServerStdio(
        params={"command": "python", "args": ["list/mcp_server.py"]},
        name="Shopping List",
        cache_tools_list=True,
    ) as shopping_server, MCPServerSse( # MCPServerSse configuration
        params={"url": "http://localhost:7000/sse"}, # Corrected: using params dict with "url" key
        name="Order Products",
        cache_tools_list=True,
    ) as order_server:
        
        agent = Agent(
            name="Shopping Assistant",
            instructions=(
                "You are an assistant that uses shopping list and order products tools to help manage a shopping list and order products.\n"
                "Your main responsibilities include:\n"
                "- Managing and updating the shopping list\n"
                "- Ordering products from appropriate suppliers\n"
                "- Handling all tasks without asking for user confirmation\n"
                "- Making decisions autonomously based on the available tools"
            ),
            mcp_servers=[shopping_server, order_server],
            model="gpt-4.1"
        )
        
        result = await Runner.run(
            starting_agent=agent,
            input="Order all the unpurchased items in the shopping list"
        )

        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```